In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import factor_analyzer as fa
import copy

tss_df = pd.read_csv("data/combined_data.csv")
items_descr = pd.read_csv("data/description_items.csv")

tss_df.drop(columns="Unnamed: 0", inplace=True)

items_descr.set_index("Unnamed: 0", drop=True, inplace=True)
items_descr.index.set_names("item_label", inplace=True)
items_descr = pd.Series(items_descr.iloc[:, 0], name="item description")

tss_df.head(3)

In [ ]:
item_labels_li = [
    'tss_coh_1', 'tss_coh_2', 'tss_coh_3', 'tss_coh_4',
    'tss_coh_5', 'tss_coh_6', 'tss_coh_7', 'tss_coh_8', 'tss_coh_9',
    'tss_coh_10', 'tss_coh_11', 'tss_coh_12', 'tss_conch_1', 'tss_conch_2',
    'tss_conch_3', 'tss_conch_4', 'tss_conch_5', 'tss_conch_6',
    'tss_conch_7', 'tss_conch_8', 'tss_cre_1', 'tss_cre_2', 'tss_cre_3',
    'tss_cre_4', 'tss_cre_5', 'tss_cre_6', 'tss_cre_7', 'tss_cre_8',
    'tss_cre_9', 'tss_cre_10', 'tss_cre_11', 'tss_cre_12', 'tss_qua_1',
    'tss_qua_2', 'tss_qua_3', 'tss_qua_4', 'tss_qua_5', 'tss_qua_6',
    'tss_qua_7', 'tss_qua_8', 'tss_rep_1', 'tss_rep_2', 'tss_rep_3',
    'tss_rep_4', 'tss_rep_5', 'tss_rep_6', 'tss_rep_7', 'tss_rep_8',
    'tss_rep_9', 'tss_rep_10', 'tss_rep_11', 'tss_rep_12', 'tss_sty_1',
    'tss_sty_2', 'tss_sty_3', 'tss_sty_4', 'tss_sty_5', 'tss_sty_6',
    'tss_sty_7', 'tss_sty_8', 'tss_sty_9', 'tss_sty_10', 'tss_sty_11',
    'tss_sty_12', 'tss_pac_1', 'tss_pac_2', 'tss_pac_3', 'tss_pac_4',
    'tss_pac_5', 'tss_pac_6', 'tss_pac_7', 'tss_pac_8', 'tss_pac_9'
]


# check if items are appropriate for EFA
# correlations mostly |.3|-|.8|
items_corr = tss_df[item_labels_li].corr()

extreme_corrs_count_dict = {}
high_corr_li = []

# print out extreme corrs
total_report = ""
for index, row in items_corr.iterrows():
    variable1 = index
    corrs = row
    i = 0
    count = 0
    for variable2, corr in corrs.iteritems():
        if (variable1 != variable2) and ((abs(corr) < 0.3) or (abs(corr) > 0.8)):
            count += 1
            if abs(corr) > 0.8:
                high_corr_li.append((variable1, variable2, corr))
        i += 1
    extreme_corrs_count_dict[variable1] = count

extreme_corrs_count = pd.Series(extreme_corrs_count_dict)
print("Descriptives for extreme correlation count:")
print(extreme_corrs_count.describe())

# uhmm that are more extreme correlations as I would like
# but maybe subscales are fairly orthogonal?

In [ ]:
# Let us look at a heatmap (absolute values of correlations)
fig, ax = plt.subplots(figsize=(20, 20))

ax = sns.heatmap(abs(items_corr),
                 xticklabels=items_corr.columns.values,
                 yticklabels=items_corr.columns.values,
                 ax=ax, cmap="viridis", vmin=0.3, vmax=0.8)

# most item clusters are just surprising orthogonal (which is actually nice)

In [ ]:
# Inspect suspicious items with low correlations here
susp_items = [
    "tss_cre_9", "tss_cre_10", "tss_cre_12",
    "tss_rep_8",
    "tss_pac_1", "tss_pac_3", "tss_pac_5", "tss_pac_6"
]

for index, row in items_corr.iterrows():
    if index in susp_items:
        print("{}: {}".format(index, items_descr[index]))
        print("Correlations >= .3:")
        if row[(row >= .3) & (row.index != index)].empty:
            print("NONE - ALL correlations are < .3!")
        else:
            print(row[(row > .3) & (row.index != index)].to_string())
        print("")

In [ ]:
# Consider excluding variables with lots correlations < .3
items_analysis_li = copy.deepcopy(item_labels_li)

excluded_items_li = [
    "tss_cre_9"
]

for item in excluded_items_li:
    items_analysis_li.remove(item)

In [ ]:
# Check for multicollinearity
print("\nHigh pairs of correlation")
for entry in high_corr_li:
    print("{} <-> {}: {}".format(entry[0], entry[1], entry[2]))
items_corr = tss_df[items_analysis_li].corr()
print("\nDeterminant of correlation matrix: {}".format(np.linalg.det(items_corr)))

# Determinant should be > .00001

In [ ]:
# Disply item descriptions for items with high corr
high_corr_items = [
    "tss_qua_1", "tss_qua_3", "tss_qua_6"
]

for item in high_corr_items:
    print(item + ":")
    print(items_descr[item] + "\n")

In [ ]:
# Decide here whether to exclude items based on high correlations
# If determinant stays high I would keep that in mind and have a look at it once a solution stabilized
# item exclusion is often easier once you have an idea of what items you need for working subscales


excluded_items_li = [
    "tss_qua_6"
]

for item in excluded_items_li:
    items_analysis_li.remove(item)

items_corr = tss_df[items_analysis_li].corr()
print("\nNew determinant of correlation matrix: {}".format(
    np.linalg.det(items_corr)))

In [ ]:
# Check Sampling Adequacy (KMO)
# (if SVD does not converge, dropna for participants with too many missing items)
# (might need to experiment how many missing are still okay)

tss_df.dropna(subset=items_analysis_li, thresh=38, inplace=True)

kmo = fa.factor_analyzer.calculate_kmo(tss_df[items_analysis_li])

print("Overall KMO: {}".format(kmo[1]))

i = 0
low_item_kmo = False
for item_kmo in kmo[0]:
    if item_kmo < .6:
        low_item_kmo = True
        item_label = item_labels_li[i]
        print("Low KMO for {} ('{}'): {}".format(
            item_label, items_descr[item_label], item_kmo))
    i += 1

if low_item_kmo == False:
    print("All item KMOs are >.6")

# Guidelines for KMO (Kaiser & Rice, 1974)
# Marvellous: values in the 0.90s
# Meritorious: values in the 0.80s
# Middling: values in the 0.70s
# Mediocre: values in the 0.60s
# Unacceptable: values in the 0.50s

In [ ]:
# Determine number of factors
# Code for parallel analysis adapted from Eric Andrews:
# https://stackoverflow.com/a/68677057


# EFA with no rotation and maximum number factors to get
efa = fa.factor_analyzer.FactorAnalyzer(rotation=None)
efa.fit(tss_df[items_analysis_li])
ev_pca, ev_efa = efa.get_eigenvalues()

# Prepare random data for parallel analysis
n, m = tss_df[items_analysis_li].shape
par_efa = fa.factor_analyzer.FactorAnalyzer(rotation=None)

# Create df to store the values
sum_par_evs = pd.DataFrame(columns=range(1, m+1))

k = 100 # 50 is probably fine, but run 100 just to be sure
# Run the fit 'k' times over a random matrix
for runNum in range(0, k):
    par_efa.fit(np.random.normal(size=(n, m)))
    ev_ser = pd.Series(par_efa.get_eigenvalues()[1], index=sum_par_evs.columns)
    sum_par_evs = sum_par_evs.append(ev_ser, ignore_index=True)
# get 95th percentile for the evs
par_95per = sum_par_evs.quantile(0.95)

In [ ]:
# OUtput become too hard to read with too many factors
# Only display a part of the EVs
facs_to_display = 15

plt.figure(figsize=(10, 6))

# Line for eigenvalue 1
plt.plot([1, facs_to_display+1], [1, 1], 'k--', alpha=0.3)
# For the random data (parallel analysis)
plt.plot(range(1, len(par_95per[:facs_to_display])+1),
         par_95per[:facs_to_display], 'b', label='EVs - random', alpha=0.4)
# Markers and line for actual EFA eigenvalues
plt.scatter(
    range(1, len(ev_efa[:facs_to_display])+1), ev_efa[:facs_to_display])
plt.plot(range(1, len(ev_efa[:facs_to_display])+1),
         ev_efa[:facs_to_display], label='EVs - survey data')

plt.title('Parallel Analysis Scree Plots', {'fontsize': 20})
plt.xlabel('Components', {'fontsize': 15})
plt.xticks(ticks=range(1, facs_to_display+1),
           labels=range(1, facs_to_display+1))
plt.ylabel('Eigenvalue', {'fontsize': 15})
plt.legend()
plt.show()

# Generate simple table with values for 95th percentile for random data and EVs for actual data

print("Factor eigenvalues for the 95th percentile of {} random matricesand for survey data for first {} factors:\n".
      format(k, facs_to_display))
print("\033[1mFactor\tEV - random data 95h perc.\tEV survey data\033[0m")

last_index = 0
last_95per_par = 0
last_ev_efa = 0
found_threshold = False

# Loop that prints previous (!) values
# if current EV from survey data is smaller than 95th percentile from random data, we reached the threshold
# in that case print the previous values in bold as it marks the number of factors determined by parallel analysis
for index, cur_ev_par in par_95per[:facs_to_display].iteritems():
    cur_ev_efa = ev_efa[index-1]
    if (index > 1) & (cur_ev_par >= cur_ev_efa) & (found_threshold == False):
        found_threshold = True
        print("\033[1m{}\t{:.2f}\t\t\t\t{:.2f}\033[0m".format(
            last_index, last_95per_par, last_ev_efa))
    elif (index > 1):
        print("{}\t{:.2f}\t\t\t\t{:.2f}".format(
            last_index, last_95per_par, last_ev_efa))

    if index == len(par_95per[:facs_to_display]):
        print("{}\t{:.2f}\t\t\t\t{:.2f}".format(index, cur_ev_par, cur_ev_efa))

    last_index = index
    last_95per_par = cur_ev_par
    last_ev_efa = cur_ev_efa
    
# determine factors to investigate based on screeplot and parallel analysis
# if both methods yield different results, examine both number of factors
# (decide for one based on whether you get a clean solution and on how interpretable the solution is)

In [ ]:
# Decide on rotation (orthogonal: Varimax, oblique: Oblimin)

# Run orthogonal rotation
efa = fa.FactorAnalyzer(n_factors=4, rotation='varimax')
efa.fit(tss_df[items_analysis_li])
# Display rotation matrix
# Shows correlation between factors before rotation (rows) and after (columns)
print("Rotation Matrix (Orthogonal Rotation):")
print(efa.rotation_matrix_)
# If matrix is non-symetrical, this indicates that an oblique rotation is called for

# Run oblique rotation
efa = fa.FactorAnalyzer(n_factors=4, rotation='oblimin')
efa.fit(tss_df[items_analysis_li])
# Display factor correlation matrix
print("\nFactor Correlation Matrix (Oblique Rotation):")
print(efa.phi_)
# If matrix has clear correlations between factors, than this indicates the need for an oblique rotation
# if in doubt use oblique

In [75]:
efa = fa.FactorAnalyzer(n_factors=4, rotation='oblimin')
efa.fit(tss_df[items_analysis_li])
# Display factor correlation matrix
efa.phi_
# If matrix has clear correlations between factors, than this indicates the need for an oblique rotation
# if in doubt use oblique

array([[ 1.        ,  0.31385736, -0.21841963,  0.27675065],
       [ 0.31385736,  1.        , -0.2166782 ,  0.10242571],
       [-0.21841963, -0.2166782 ,  1.        , -0.18944231],
       [ 0.27675065,  0.10242571, -0.18944231,  1.        ]])

In [85]:
# Decide on rotation (orthogonal: Varimax, oblique: Oblimin)
efa = fa.FactorAnalyzer(n_factors=4, rotation='varimax')
efa.fit(tss_df[items_analysis_li])

comm = pd.DataFrame(efa.get_communalities(
), index=tss_df[items_analysis_li].columns, columns=['Communalities'])
comm.sort_values("Communalities", ascending=False)

,Communalities
tss_rep_5,0.728622
tss_rep_6,0.694061
tss_qua_3,0.691064
tss_rep_4,0.683272
tss_rep_10,0.678602
...,...
tss_sty_8,0.268446
tss_pac_9,0.256971
tss_conch_5,0.242766
tss_conch_2,0.216062


In [260]:
efa.rotation_matrix_

array([[ 0.84385341,  0.45015965, -0.26388026, -0.12503967],
       [-0.44516004,  0.84702851, -0.10662776,  0.27019578],
       [ 0.20771225,  0.19707984,  0.9528539 ,  0.1004221 ],
       [ 0.21586935, -0.20262906, -0.10520062,  0.9493549 ]])

In [86]:
loadings = pd.DataFrame(efa.loadings_, index=tss_df[items_analysis_li].columns)
loadings["descr"] = loadings.apply(lambda x: items_descr[x.name], axis=1)

loadings.sort_values(0, key=abs, ascending=False)

,0,1,2,3,descr
tss_coh_2,-0.777845,0.054679,-0.002189,-0.208441,I had a hard time recognizing the thread of th...
tss_coh_1,-0.771212,0.180552,-0.067479,-0.116311,I had a hard time making sense of what was goi...
tss_coh_10,0.751422,-0.017725,0.030604,0.114469,The story had a clearly identifiable plot.
tss_coh_8,0.744648,0.072483,0.078488,0.083257,The story stayed on topic with a consistent plot.
tss_conch_6,-0.734413,0.200740,-0.121740,-0.022294,The behavior of characters in the story seemed...
...,...,...,...,...,...
tss_sty_5,0.025351,0.024296,0.604747,0.083756,The story used complex vocabulary.
tss_cre_7,0.024875,0.112235,0.356775,0.607638,There were interesting twists and turns in the...
tss_rep_7,-0.024673,0.767662,0.021190,-0.142404,Characters repeated their actions with little ...
tss_rep_12,0.015071,0.692873,-0.204989,-0.063517,Particular words were used too often in the st...


In [235]:
mask_good_item = ((abs(loadings[[0, 1, 2, 3]])
                   >= .32).astype(int).sum(axis=1)) == 1
items_analysis_li = mask_good_item[mask_good_item].index

# rinse and repeat...